In [1]:
""" Convert RDS table to csv files on S3 and GCS
-------------------------------------------------------------------------------

update 2020/02/10 update output 11-12 input 8-9

Author: Rutger Hofste
Date: 20180712
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04
"""

# imports
import re
import os
import numpy as np
import pandas as pd
from retrying import retry
from datetime import timedelta
from sqlalchemy import *
pd.set_option('display.max_columns', 500)
import multiprocessing

SCRIPT_NAME = 'Y2018M07D17_RH_RDS_To_S3_V02'
OUTPUT_VERSION = 14

TESTING = 0

DATABASE_ENDPOINT = "aqueduct30v05.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com"
DATABASE_NAME = "database01"

INPUT_TABLE_NAME = "y2018m07d30_rh_coalesce_columns_v01_v09"

ec2_output_path = "/volumes/data/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
gcs_output_path = "gs://aqueduct30_v01/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

print("Input Table: " , INPUT_TABLE_NAME,
      "\nOutput ec2: ", ec2_output_path,
      "\nOutput s3: " , s3_output_path,
      "\nOutput gcs: ", gcs_output_path)

Input Table:  y2018m07d30_rh_coalesce_columns_v01_v09 
Output ec2:  /volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14/ 
Output s3:  s3://wri-projects/Aqueduct30/processData/Y2018M07D17_RH_RDS_To_S3_V02/output_V14/ 
Output gcs:  gs://aqueduct30_v01/Y2018M07D17_RH_RDS_To_S3_V02/output_V14/


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2020M02D19 UTC 14:13


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
!rm -r {ec2_output_path}
!mkdir -p {ec2_output_path}

rm: cannot remove '/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14/': No such file or directory


In [4]:
F = open("/.password","r")
password = F.read().splitlines()[0]
F.close()

engine = create_engine("postgresql://rutgerhofste:{}@{}:5432/{}".format(password,DATABASE_ENDPOINT,DATABASE_NAME))

In [5]:
cpu_count = multiprocessing.cpu_count()
print("Power to the maxxx:", cpu_count)

Power to the maxxx: 16


In [6]:
sql = "SELECT DISTINCT pfafid_30spfaf06 FROM {} ORDER BY pfafid_30spfaf06".format(INPUT_TABLE_NAME)

In [7]:
df = pd.read_sql(sql,engine)

In [ ]:
df.shape

(16385, 1)

In [ ]:
df.head()

,pfafid_30spfaf06
0,111011
1,111012
2,111013
3,111014
4,111015


In [ ]:
if TESTING:
    df = df[0:10]

In [ ]:
df_split = np.array_split(df, cpu_count*100)

In [ ]:
def basin_to_csv(df):
    for index, row in df.iterrows():
        pfafid = row["pfafid_30spfaf06"]
        sql = "SELECT * FROM {} WHERE pfafid_30spfaf06 = {}".format(INPUT_TABLE_NAME,pfafid)
        df_basin = pd.read_sql(sql,engine)
        now = datetime.datetime.now()
        df_basin["processed_timestamp"] = pd.Timestamp(now)  
        output_file_name = "{}_V{:02.0f}.csv".format(pfafid,OUTPUT_VERSION)
        output_file_path = "{}/{}".format(ec2_output_path,output_file_name)
        # Added on Y2018M07D30, convert to numeric type if possible
        df_basin2 = df_basin.apply(pd.to_numeric,errors='ignore')
        df_basin2.to_csv(output_file_path)
        print(output_file_path)

In [ ]:
# cleared output to save space
p= multiprocessing.Pool()
results_buffered = p.map(basin_to_csv,df_split)
p.close()
p.join()

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//155562_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//223940_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//115801_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//232144_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//132895_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//172497_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//111011_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//122784_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//142625_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//162203_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//131902_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//181976_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//157807_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//145102_V14.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//131908_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//127450_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//172524_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//132930_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//224003_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//111019_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//155572_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//115809_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//122810_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//142633_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//158116_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//162300_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//145109_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//181993_V14.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//132160_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//111045_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//225111_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//122845_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//116006_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//162444_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//183000_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//232250_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//172542_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//153035_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//158143_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//145207_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//127468_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//155580_V14.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//117102_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//225118_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//142649_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//162461_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//132955_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//158211_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//155617_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//211015_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//172549_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//132224_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//232303_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//153043_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//127485_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//122854_V14.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//172557_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//145500_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//162468_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//211032_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//127493_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//153050_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//132241_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//232401_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//155635_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//122870_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//132973_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//142658_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//158220_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//111088_V14.csv
/volumes/data/Y2018M

...

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//742429_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//722809_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//642463_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//753301_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//911107_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//782696_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//662809_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//812109_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//712601_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//652209_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//822479_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//831970_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//742430_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//742987_V14.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//822496_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//742447_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//662907_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//742995_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//772348_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//911219_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//652306_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//726016_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//723071_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//753309_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//832106_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//642472_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//712609_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//812208_V14.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//742482_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//723078_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//753407_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//642479_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//743005_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//822515_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//713221_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//911267_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//782803_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//812306_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//664024_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//652405_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//772402_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//742483_V14.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//753506_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//832430_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//726077_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//911288_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//782910_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//812450_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//713229_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//822542_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//751203_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//664043_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//911289_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//832441_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//723116_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//652503_V14.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//772608_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//832450_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//753624_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//812478_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//751402_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//822560_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//726087_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//742510_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//782993_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//772609_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//832460_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//753625_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//812479_V14.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//751403_V14.csv
/volumes/data/Y2018M

In [ ]:
!aws s3 cp {ec2_output_path} {s3_output_path} --recursive --quiet

In [ ]:
# cleared output to save space
!gsutil -m cp \
{ec2_output_path}/*.csv \
{gcs_output_path}

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//111011_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//111012_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//111013_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//111015_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//111014_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//111016_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//111017_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//111018_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//111019_V14.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//112170_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//112130_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//112160_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//112150_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//112047_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//112180_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//112400_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//112190_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//112200_V14.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//114443_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//114444_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//114442_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//114445_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//114446_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//114447_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//114448_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//114449_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//114450_V14.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//115423_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//115424_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//115425_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//115426_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//115427_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//115428_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//115429_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//115430_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//115440_V14.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//117103_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//117104_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//117105_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//117106_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//117107_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//117108_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//117109_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//117201_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//117202_V14.csv [Content-Type=text/

...

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//862098_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//862099_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//863000_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//863001_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//863002_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//863003_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//863004_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//863005_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//863006_V14.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//911290_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//911300_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//911401_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//911402_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//911403_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//911404_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//911405_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//911406_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//911407_V14.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//912204_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//912205_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//912207_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//912206_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//912208_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//912209_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//912300_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//912400_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//912510_V14.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//913476_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//913477_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//913478_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//913479_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//913480_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//913490_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//913500_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//913510_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//913520_V14.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//914507_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//914508_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//914509_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//914600_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//914700_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//914701_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//914702_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//914703_V14.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V14//914704_V14.csv [Content-Type=text/

In [ ]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:39:32.043603


Previous runs:  
0:42:01.416597  
0:42:14.565295  
0:31:40.728932  
0:23:35.132388  
0:30:37.392875

